Лабораторная работа
Подготовка обучающей и тестовой выборки, кросс-валидация и подбор гиперпараметров на примере метода ближайших соседей.

Цель лабораторной работы: изучение способов подготовки выборки и подбора гиперпараметров на примере метода ближайших соседей.
Требования к отчету:

Отчет по лабораторной работе должен содержать:

    титульный лист;
    описание задания;
    текст программы;
    экранные формы с примерами выполнения программы.

Задание:

    Выберите набор данных (датасет) для решения задачи классификации или регрессии.
    В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
    С использованием метода train_test_split разделите выборку на обучающую и тестовую.
    Обучите модель ближайших соседей для произвольно заданного гиперпараметра K. Оцените качество модели с помощью подходящих для задачи метрик.
    Произведите подбор гиперпараметра K с использованием GridSearchCV и RandomizedSearchCV и кросс-валидации, оцените качество оптимальной модели. Используйте не менее двух стратегий кросс-валидации.
    Сравните метрики качества исходной и оптимальной моделей.


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Генерация случайного датасета (замени на свой CSV)
np.random.seed(42)
df = pd.read_csv("coffee_shop_sales.csv").sample(frac=0.03, random_state=42)


# 1. Создаём целевой признак на основе медианы transaction_qty
threshold = df["transaction_qty"].median()
df["High_Transaction"] = (df["transaction_qty"] > threshold).astype(int)

# 2. Проверяем баланс классов
print("Баланс классов:\n", df["High_Transaction"].value_counts(normalize=True))

# 3. Убираем утечки данных (transaction_qty не должен быть в X)
X = df[['store_id', 'product_id', 'unit_price', 'Total_Bill', 'Hour', 'Month', 'Day of Week']]
y = df['High_Transaction']

# 4. Масштабируем числовые признаки
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. StratifiedShuffleSplit для сбалансированного разбиения
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# 6. Обучаем KNN с K=5
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

# 7. Метрики для K=5
print("Метрики качества для K=5:")
print(classification_report(y_test, y_pred))

# 8. Подбор оптимального K через RandomizedSearchCV
param_dist = {'n_neighbors': np.arange(1, 21)}
grid_search = GridSearchCV(KNeighborsClassifier(), param_dist, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Лучшее значение K по GridSearchCV:", grid_search.best_params_)

# 9. Подбор оптимального K через RandomizedSearchCV
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
best_k = random_search.best_params_['n_neighbors']
print(f"Лучшее значение K по RandomizedSearchCV: {best_k}")

# Кросс-валидация KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf_scores = cross_val_score(KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors']), X_train, y_train, cv=kf, scoring='accuracy')
print("Средняя точность по KFold:", np.mean(kf_scores))

# Кросс-валидация с StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf_scores = cross_val_score(KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors']), X_train, y_train, cv=skf, scoring='accuracy')
print("Средняя точность по StratifiedKFold:", np.mean(skf_scores))

# 10. Обучаем модель с оптимальным K
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train, y_train)
y_best_pred = best_knn.predict(X_test)

# 11. Метрики для лучшей модели
print("Метрики качества для оптимального K:")
print(classification_report(y_test, y_best_pred))


Баланс классов:
 High_Transaction
0    0.586631
1    0.413369
Name: proportion, dtype: float64
Метрики качества для K=5:
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       525
           1       0.74      0.68      0.71       370

    accuracy                           0.77       895
   macro avg       0.76      0.76      0.76       895
weighted avg       0.77      0.77      0.77       895

Лучшее значение K по GridSearchCV: {'n_neighbors': np.int64(17)}
Лучшее значение K по RandomizedSearchCV: 17
Средняя точность по KFold: 0.7593671133335939
Средняя точность по StratifiedKFold: 0.7705367816541002
Метрики качества для оптимального K:
              precision    recall  f1-score   support

           0       0.78      0.88      0.83       525
           1       0.79      0.66      0.72       370

    accuracy                           0.79       895
   macro avg       0.79      0.77      0.77       895
weighted avg       0.79      0.7